In [1]:
from peft import (LoraConfig,IA3Config, PeftType, PrefixTuningConfig,
                  PromptEncoderConfig, PromptTuningConfig, TaskType,
                  get_peft_config, get_peft_model, get_peft_model_state_dict,
                  prepare_model_for_int8_training,
                  prepare_model_for_kbit_training, set_peft_model_state_dict)
from scipy.special import softmax
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (AutoModelForSequenceClassification,
                          AutoModelForTokenClassification, AutoTokenizer,
                          DataCollatorForTokenClassification,
                          LlamaForSequenceClassification, LlamaTokenizer,
                          Trainer, TrainingArguments,
                          get_linear_schedule_with_warmup, set_seed)
from model_utils import count_trainable_parameters, get_model_size, get_full_model_size
import pandas as pd
from tqdm import tqdm
import yaml
# import sys and append path
import sys
sys.path.append("../")
from peft_trainer import create_peft_config
from loguru import logger as loguru_logger
from argparse import Namespace

In [2]:

arg_dict = {
  "log_save_dir": "/mnt/sdc/niallt/saved_models/peft_training/logs",
  "task_type": "SEQ_CLS",
  "peft_method": "LORA",
  "lora_rank": 8, 
  "lora_alpha": 16,
  "lora_dropout": 0.1,
  "learning_rate": 3e-4,
  "num_virtual_tokens": 10 
}

# convert above args to a namespace object

args = Namespace(**arg_dict)

In [3]:
# model = AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-2-7b-hf")

In [4]:
# model

In [5]:
# create args


In [10]:


model_type_mappings = {"vanilla": "roberta-base",
               "mobile": "nlpie/bio-mobilebert",
               "distil": "nlpie/distil-biobert",
               "tiny": "nlpie/tiny-biobert",
               "llama-7b": "meta-llama/Llama-2-7b-hf",
               }

peft_types = ["PROMPT_TUNING","LORA", "PREFIX_TUNING", "P_TUNING"]

def get_number_of_trainable_params(model_type_mappings:dict,
                                   peft_types:list,
                                   task_type:str = "SEQ_CLS",
                                   num_labels:int = 2):

    # set up empty dicts to full for dfs
    model_peft_dict = {}
    
    for model_type in model_type_mappings.keys():
        
        model_dict = {}
        model_name_or_path = model_type_mappings[model_type]
        model_args = dict(pretrained_model_name_or_path=model_name_or_path, 
                          num_labels=num_labels, 
                          output_hidden_states=False, 
                          trust_remote_code=True)

            
        if task_type == "SEQ_CLS":
            model = AutoModelForSequenceClassification.from_pretrained(**model_args)
        elif task_type == "TOKEN_CLS":
            model = AutoModelForTokenClassification.from_pretrained(**model_args)
        
        # falcon model seems to use model config to define pad token and the remote code panicks if you don't set it
        if "falcon" in model_name_or_path:
            model.config.use_cache = False            

        # count total trainable params before peft
        total_trainable_params = count_trainable_parameters(model)
        
        # get model size and full model size too
        model_size_MB, model_size_GB = get_model_size(model)
        full_model_size_MB, full_model_size_GB = get_full_model_size(model)
        
        for peft_method in tqdm(peft_types, desc=f"model type: {model_type}"):
            
            
            # set up some PEFT params
            peft_config, lr = create_peft_config(args, peft_method, model_name_or_path,task_type)
            model = get_peft_model(model, peft_config)
            print(f"peft config is: {peft_config}")
            # print(model)
            model.print_trainable_parameters()
            
            # lets also confirm this directly and save to args
            n_trainable_params = count_trainable_parameters(model)
            # proportion of total trainable params
            n_trainable_params_perc = (n_trainable_params / total_trainable_params) * 100
            
            # get size of peft adapter only
            peft_model_size_MB, peft_model_size_GB = get_model_size(model)
            peft_full_model_size_MB, peft_full_model_size_GB = get_full_model_size(model)
            
            # store the model name, peft method and number of trainable params
            model_dict[peft_method] = {"n_trainable_params": n_trainable_params,
                                 "total_trainable_params": total_trainable_params,
                                 "n_trainable_params_perc": n_trainable_params_perc,
                                 "model_size_MB": model_size_MB,
                                 "model_size_GB": model_size_GB,
                                 "full_model_size_MB": full_model_size_MB,
                                 "full_model_size_GB": full_model_size_GB,
                                 "peft_model_size_MB": peft_model_size_MB,
                                 "peft_model_size_GB": peft_model_size_GB,
                                 "peft_full_model_size_MB": peft_full_model_size_MB,
                                 "peft_full_model_size_GB": peft_full_model_size_GB,}
            
        model_peft_dict[model_type] = model_dict

    return model_peft_dict

In [11]:
all_df = get_number_of_trainable_params(model_type_mappings, peft_types)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should pr

Model size (MB): 475.49121856689453
Total size (MB): 951.0699663162231


model type: vanilla:   0%|          | 0/4 [00:00<?, ?it/s]2023-10-06 11:29:05.025 | INFO     | peft_trainer:create_peft_config:672 - Using PROMPT_TUNING


peft config is: PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path='roberta-base', revision=None, task_type='SEQ_CLS', inference_mode=False, num_virtual_tokens=10, token_dim=768, num_transformer_submodules=1, num_attention_heads=12, num_layers=12, prompt_tuning_init=<PromptTuningInit.RANDOM: 'RANDOM'>, prompt_tuning_init_text=None, tokenizer_name_or_path=None)
trainable params: 1,191,940 || all params: 125,246,980 || trainable%: 0.9516716490888643
Model size (MB): 4.5468902587890625


model type: vanilla:  25%|██▌       | 1/4 [00:00<00:01,  1.67it/s]2023-10-06 11:29:05.626 | INFO     | peft_trainer:create_peft_config:615 - Using LORA


...

model type: llama-7b:  50%|█████     | 2/4 [01:18<01:22, 41.07s/it]2023-10-06 11:32:02.976 | INFO     | peft_trainer:create_peft_config:666 - Using PREFIX_TUNING


peft config is: PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, num_virtual_tokens=10, token_dim=4096, num_transformer_submodules=1, num_attention_heads=32, num_layers=32, encoder_hidden_size=4096, prefix_projection=False)
trainable params: 2,621,440 || all params: 6,614,175,744 || trainable%: 0.039633661116096286
Model size (MB): 10.0
Total size (MB): 25369.347331047058


model type: llama-7b:  75%|███████▌  | 3/4 [01:49<00:36, 36.36s/it]2023-10-06 11:32:33.734 | INFO     | peft_trainer:create_peft_config:686 - Using P_TUNING


peft config is: PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, num_virtual_tokens=10, token_dim=4096, num_transformer_submodules=1, num_attention_heads=32, num_layers=32, encoder_reparameterization_type=<PromptEncoderReparameterizationType.MLP: 'MLP'>, encoder_hidden_size=128, encoder_num_layers=2, encoder_dropout=0.0)
trainable params: 1,110,272 || all params: 6,612,664,576 || trainable%: 0.016790084953493944
Model size (MB): 4.2353515625
Total size (MB): 25357.835852622986


model type: llama-7b: 100%|██████████| 4/4 [02:21<00:00, 35.47s/it]


In [12]:
all_df

{'vanilla': {'PROMPT_TUNING': {'n_trainable_params': 1191940,
   'total_trainable_params': 124647170,
   'n_trainable_params_perc': 0.956251152753809,
   'model_size_MB': 475.49121856689453,
   'model_size_GB': 0.46434689313173294,
   'full_model_size_MB': 951.0699663162231,
   'full_model_size_GB': 0.9287792639806867,
   'peft_model_size_MB': 4.5468902587890625,
   'peft_model_size_GB': 0.004440322518348694,
   'peft_full_model_size_MB': 482.42061138153076,
   'peft_full_model_size_GB': 0.47111387830227613},
  'LORA': {'n_trainable_params': 294912,
   'total_trainable_params': 124647170,
   'n_trainable_params_perc': 0.23659742936803138,
   'model_size_MB': 475.49121856689453,
   'model_size_GB': 0.46434689313173294,
   'full_model_size_MB': 951.0699663162231,
   'full_model_size_GB': 0.9287792639806867,
   'peft_model_size_MB': 1.125,
   'peft_model_size_GB': 0.0010986328125,
   'peft_full_model_size_MB': 480.1683073043823,
   'peft_full_model_size_GB': 0.46891436260193586},
  'PREFI

In [13]:
# write to file
with open("../model_type_trainable_model_size.yaml", "w") as f:
    yaml.dump(all_df, f)